In [1]:
from dataclasses import dataclass
from functools import reduce
from pathlib import Path

data_file = Path("../Data/day5.txt").read_text()


EXAMPLE = """47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47"""


@dataclass
class PrinterPageOrderRule:
    X: int
    Y: int


@dataclass
class PrinterData:
    page_order_rules: list[PrinterPageOrderRule]
    updates: list[list[int]] | None


def prepare(input: str):
    def group_line(data: PrinterData, line: str):
        if line == "" and data.updates is None:
            data.updates = []

            return data

        if data.updates is not None:
            data.updates.append(list(map(int, line.split(","))))

            return data

        page_order_rule = line.split("|")

        data.page_order_rules.append(
            PrinterPageOrderRule(int(page_order_rule[0]), int(page_order_rule[1]))
        )

        return data

    return reduce(
        group_line, input.splitlines(), PrinterData(page_order_rules=[], updates=None)
    )


def map_rules(data: PrinterData):
    mapped_rules: dict[str, dict[str, list[PrinterPageOrderRule]]] = {"X": {}, "Y": {}}
    for rule in data.page_order_rules:
        try:
            mapped_rules["X"][str(rule.X)].append(rule)
        except KeyError:
            mapped_rules["X"][str(rule.X)] = [rule]

        try:
            mapped_rules["Y"][str(rule.Y)].append(rule)
        except KeyError:
            mapped_rules["Y"][str(rule.Y)] = [rule]

    return mapped_rules


def check_update_is_in_right_order(
    update: list[int], mapped_rules: dict[str, dict[str, list[PrinterPageOrderRule]]]
):
    update_length = len(update)
    for i in range(update_length):
        item = update[i]
        for j in range(i + 1, update_length):
            next_update = update[j]
            y_rules = mapped_rules["Y"].get(str(next_update))
            if y_rules is None:
                return False

            rule_found: PrinterPageOrderRule | None = None
            for rule in y_rules:
                if rule.X != item or rule.Y != next_update:
                    continue

                rule_found = rule

            if rule_found is None:
                return False

    return True


data = prepare(data_file)
example_data = prepare(EXAMPLE)

In [2]:
def part1(data: PrinterData):
    assert data.updates is not None

    mapped_rules = map_rules(data)
    valid_update_middle_numbers: list[int] = []
    for update in data.updates:
        update_is_in_right_order = check_update_is_in_right_order(
            update=update, mapped_rules=mapped_rules
        )
        if not update_is_in_right_order:
            continue

        valid_update_middle_numbers.append(update[len(update) // 2])

    return sum(valid_update_middle_numbers)


assert part1(example_data) == 143

result = part1(data)

print("result is", result)

result is 4135


In [ ]:
def sort_update(
    update: list[int], mapped_rules: dict[str, dict[str, list[PrinterPageOrderRule]]]
):
    update_length = len(update)
    for i in range(update_length):
        item = update[i]
        rules = mapped_rules["X"].get(str(item))
        for j in range(i + 1, update_length):
            next_update = update[j]
            rule_found: PrinterPageOrderRule | None = None
            for rule in rules:
                if rule.X != item or rule.Y != next_update:
                    continue

                rule_found = rule

            if rule_found is None:
                continue

            update[i], update[j] = update[j], update[i]

    return update


def part2(data: PrinterData):
    assert data.updates is not None

    mapped_rules = map_rules(data)
    invalid_updates: list[list[int]] = []
    for update in data.updates:
        update_is_in_right_order = check_update_is_in_right_order(
            update=update, mapped_rules=mapped_rules
        )
        if update_is_in_right_order:
            continue

        invalid_updates.append(update)

    print(invalid_updates)


part2(example_data)

part2(data)

[[75, 97, 47, 61, 53], [61, 13, 29], [97, 13, 75, 29, 47]]
[[79, 16, 83, 58, 33, 35, 29, 52, 45, 61, 17, 54, 41, 12, 26], [74, 49, 31, 72, 95, 27, 44, 23, 91, 89, 98, 67, 94, 18, 48], [74, 44, 15, 48, 24, 18, 55, 49, 36, 38, 13, 89, 67, 52, 41, 31, 23, 43, 95, 98, 21], [67, 94, 49, 55, 15, 98, 58, 95, 74, 87, 18, 21, 72], [17, 89, 95, 36, 42, 91, 62, 75, 79, 74, 87, 23, 26, 86, 78, 29, 35, 54, 84, 58, 21], [53, 79, 37, 74, 62, 58, 95, 89, 42, 12, 16, 17, 75, 86, 26, 36, 35], [13, 84, 36, 23, 58, 48, 95, 89, 26, 55, 24, 49, 15, 87, 37, 21, 43], [48, 21, 13, 67, 23, 98, 49, 18, 84], [58, 43, 89, 95, 23, 49, 72], [44, 23, 49, 37, 98, 31, 38, 95, 89, 74, 13, 67, 27], [45, 74, 18, 19, 38, 23, 43, 67, 24, 48, 21, 98, 52, 94, 41, 49, 31, 83, 44, 72, 13], [91, 62, 79, 16, 42, 45, 26, 19, 36, 86, 53, 17, 54, 35, 37], [13, 52, 45, 19, 16, 12, 72, 24, 78], [45, 17, 31, 84, 42, 78, 19, 52, 26, 61, 83, 29, 88, 12, 62], [33, 87, 88, 75, 16, 58, 42, 37, 78, 79, 62], [61, 48, 41, 43, 94, 31, 55, 21, 9